In [17]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import numpy as np

In [19]:
df = pd.read_csv('master.csv')
df = pd.read_csv('master.csv')
df = df.iloc[2000:5151]
df = df[["Coding","Citation"]]
df = df[pd.to_numeric(df["Coding"], errors="coerce").notna()]
df["Coding"] = df["Coding"].astype(int)
df = df.rename(columns={"Citation": "text", "Coding": "label"})
df.shape

/var/folders/gt/3sb0875538n39l60gwltp88w0000gn/T/ipykernel_68117/2236956213.py:1: DtypeWarning: Columns (1,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('master.csv')
/var/folders/gt/3sb0875538n39l60gwltp88w0000gn/T/ipykernel_68117/2236956213.py:2: DtypeWarning: Columns (1,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('master.csv')


(1629, 2)

In [21]:
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df["label"])
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 5. Load tokenizer and tokenize
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# 6. Load model
num_labels = df["label"].nunique()
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=num_labels)

# 7. Training config
training_args = TrainingArguments(
    output_dir="./deberta_citation_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

Map:   0%|          | 0/1384 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
trainer.train()

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.543411
2,No log,0.526421
3,0.430000,0.701256


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=519, training_loss=0.4258346869996058, metrics={'train_runtime': 372.5776, 'train_samples_per_second': 11.144, 'train_steps_per_second': 1.393, 'total_flos': 318255664416768.0, 'train_loss': 0.4258346869996058, 'epoch': 3.0})

In [25]:
y_true = np.array(val_dataset["label"])
preds_output = trainer.predict(val_dataset)
y_pred = np.argmax(preds_output.predictions, axis=1)

print(classification_report(y_true, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.87      0.86      0.86       188
           3       0.53      0.70      0.61        44
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         7

    accuracy                           0.79       245
   macro avg       0.28      0.31      0.29       245
weighted avg       0.76      0.79      0.77       245



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
